# Previsão de casos com Redes Neurais

## Preprocessamento da base de dados Dataset_finalizado_filtrado.csv.

In [100]:
import pandas as pd 

base = pd.read_csv('Dataset_finalizado_filtrado.csv').dropna()

Abaixo temos a separação da base de dados entre variáveis previsores (X) e previsões (Y).

In [101]:
X = base.iloc[:, :].drop(columns=['last_available_confirmed']).values
Y = base.iloc[:, 2:3].values

Logo em seguida fazemos a tranformação das variáveis categóricas em variáveis numéricas através do LabelEncoder. Tranformamos cada cidade em uma coluna, para que esse valor não interfira na previsão, através do OneHotEncoder e ColumnTransformer. Por fim padronizamos os valores para que nenhuma variável seja mais significante que a outra no início, deixando o algoritmo de redes neurais definir os seus pesos.

In [102]:
from sklearn.preprocessing import LabelEncoder

labelencoder_X = LabelEncoder()
X[:,0] = labelencoder_X.fit_transform(X[:,0])
X[:,1] = labelencoder_X.fit_transform(X[:,1])

In [103]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [1])], remainder='passthrough')
X = onehotencorder.fit_transform(X).toarray()

In [104]:
from sklearn.preprocessing import StandardScaler

scaler_x = StandardScaler()
X = scaler_x.fit_transform(X)
scaler_y=StandardScaler()
Y = scaler_y.fit_transform(Y)

Para concluir o nosso preprocessamento, vamos dividir as variáveis X e Y em variáveis de treinamento e teste, usando o train_test_split.

In [105]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state=0)

## Treinamento do Regressor

In [106]:
from sklearn.neural_network import MLPRegressor
regressor = MLPRegressor(hidden_layer_sizes=(154,154,154,154))
regressor.fit(X_train, Y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(154, 154, 154, 154), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)

In [107]:
score = regressor.score(X_train, Y_train)
print('R² =', score)

R² = 0.9634741007332404


## Fazendo previsões com nosso regressor para o conjunto de teste

In [108]:
previsoes = regressor.predict(X_test)

Para comparar os erros da nossa previsão, vamos primeiro tirar o escalonamento das váriáveis Y_test e Previsoes.

In [109]:
Y_test = scaler_y.inverse_transform(Y_test)
previsoes = scaler_y.inverse_transform(previsoes)

In [110]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test, previsoes)
print('Média do Erro Absoluto:', mae)

Média do Erro Absoluto: 12.979040807421251
